In [ ]:
#This is the main Jupyter Note book that walks through the entire process of how I came about my Code.
# I will not run these cells so they wont interfere with the notbooks I already set up.

In [ ]:
#Importing Packages
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm 
from statsmodels.stats.outliers_influence import variance_inflation_factor
import datetime as dt
import pandas as pd
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import r2_score
from numpy import loadtxt
from xgboost import XGBClassifier
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '%.6f' % x)



In [ ]:
# Importing StockX Data Set into varible
sneakerData = pd.read_csv('data/StockX-Data-Contest-2019-3.csv')

In [ ]:
#Data Cleaning Cell

#Making Sure all Sneaker Names are lowercase to be on the Same accord.
sneakerData['Sneaker Name'] = sneakerData['Sneaker Name'].str.lower()

#Cleaning the dates column
sneakerData['Order Date'] = sneakerData['Order Date'].astype('datetime64[ns]')
sneakerData['Release Date'] = sneakerData['Release Date'].astype('datetime64[ns]')
sneakerData['Turnover Days'] = sneakerData['Order Date'] - sneakerData['Release Date']

#Removing the $ and , From Sale Price and Retial Price to be able to turn into intergers for the future
sneakerData['Sale Price'] =  sneakerData['Sale Price'].astype(str).str.replace('$', '')
sneakerData['Sale Price'] =  sneakerData['Sale Price'].astype(str).str.replace(',', '')
sneakerData['Retail Price'] =  sneakerData['Retail Price'].astype(str).str.replace('$', '')
sneakerData['Retail Price'] =  sneakerData['Retail Price'].astype(str).str.replace(',', '')

# Making Sales and Retial Price into Int
sneakerData['Sale Price'] = sneakerData['Sale Price'].astype(int)
sneakerData['Retail Price'] = sneakerData['Retail Price'].astype(int)

In [ ]:
#Adding the Profit Column to possibly compare how future features effects the profitability of the shoe.
sneakerData['Profit'] = sneakerData['Sale Price'] - sneakerData['Retail Price']
sneakerData['Profit Ratio'] = (sneakerData['Profit'] / sneakerData['Retail Price']).round(2)

In [ ]:
#Configuring and adding the Model Column to do Comparisons by Modeling.
sneakerData["Model"] = sneakerData['Sneaker Name'].apply(
    lambda x : 'yeezy-boost-350' if 'yeezy' in x.split("-") else (
        'air-jordan-1-retro-high' if 'jordan' in x.split('-') else (
            'air-force-1' if 'force' in x.split('-') else(
                'air-max-90' if '90' in x.split('-') else (
                    'air-max-97' if '97' in x.split('-') else (
                        'air-presto' if 'presto' in x.split('-') else (
                            'air-vapormax' if 'vapormax' in x.split('-') else (
                                'blazer-mid' if 'blazer' in x.split('-') else (
                                    'react-hyperdunk-2017-flyknit' if 'hyperdunk' in x.split('-') else (
                                        'zoom-fly' if 'zoom' in x.split('-') else (np.nan)
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )
)

In [ ]:
#check if all the Sneaker Names are categorised into Model Variants
uncategorised_model = pd.DataFrame()
uncategorised_model[sneakerData['Model'].isnull() == True]
print(uncategorised_model)

In [ ]:
#importing the colour information from stockX website, based on each sneaker name
colourway = pd.read_excel('data/supplemental_data_colorway.xlsx')
colourway['Style'] = colourway['Style'].str.lower()
colourway.head()

#merging the colourway DF with the sneakerData DF
sneakerData = pd.merge(sneakerData,colourway,how='left',left_on='Sneaker Name',right_on='Style')

In [ ]:
#Filling the Null values of Color columns with 0s to be able to self One-Hot-Encode Colors
sneakerData['Black'] = sneakerData['Black'].fillna(0)
sneakerData['White'] = sneakerData['White'].fillna(0)
sneakerData['Green'] = sneakerData['Green'].fillna(0)
sneakerData['Neo'] = sneakerData['Neo'].fillna(0)
sneakerData['Orange'] = sneakerData['Orange'].fillna(0)
sneakerData['Tan/Brown'] = sneakerData['Tan/Brown'].fillna(0)
sneakerData['Pink'] = sneakerData['Pink'].fillna(0)
sneakerData['Blue'] = sneakerData['Blue'].fillna(0)
sneakerData['Colorful'] = sneakerData['Colorful'].fillna(0)

In [ ]:
#Dropping Number of Sales, Website,Product Line,
# and Buyer region to because I wanted to focus on the specific shoe features that could possibily be features.

parsed_data = sneakerData.drop(['Number of Sales','Website','Product Line','Buyer Region'],axis=1)
parsed_data['Turnover Days'] = parsed_data['Turnover Days'].dt.days
parsed_data['Turnover Weeks'] = (parsed_data['Turnover Days'] / 7).round(0)
parsed_data = parsed_data.drop(parsed_data[parsed_data['Turnover Weeks'] < 0].index)
parsed_data = parsed_data.drop(parsed_data[parsed_data['Turnover Weeks'] > 52].index)

#consolidating the primary colors into 1 single column
def get_col(row):
    for color in parsed_data.columns[10:20]:
        if row[color] == 1:
            return color
parsed_data['color'] = parsed_data.apply(get_col, axis=1)
parsed_data = parsed_data.replace(0, np.nan)


In [ ]:
#Saving Clean Data into new features.
parsed_data.to_csv('parsed_data.csv')